In [1]:
import torch_geometric.nn as gnn
import torch_geometric
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.linalg
from torch_geometric.nn import GCNConv
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
# from torch_geometric.datasets import Planetoid
# name_data = 'Cora'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Wisconsin'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import Planetoid
# name_data = 'CiteSeer'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Cornell'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Texas'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WikipediaNetwork
# name_data = 'Chameleon'
# dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

from torch_geometric.datasets import WikipediaNetwork
name_data = 'Squirrel'
dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

In [4]:
data = dataset[0]

edges = data.edge_index

#dimension: num_nodes * num_features
features = data.x

num_nodes = features.shape[0]
num_features = features.shape[1]
#dimension: num_nodes * num_nodes
adj = torch.zeros((num_nodes,num_nodes))
degree = torch.zeros((num_nodes,num_nodes))
for i in range(edges.shape[1]):
    first = edges[0][i]
    second = edges[1][i]
    adj[first][second] = 1
    adj[second][first] = 1
for i in range(num_nodes):
    degree[i][i] = sum(adj[i][:])

In [5]:
sample_num = int(num_nodes*0.6)
sample_lst = np.random.choice(num_nodes,sample_num, replace = False)
train_mask = [False] * num_nodes
test_mask = [True] * num_nodes
for i in sample_lst:
  train_mask[i] = True
  test_mask[i] = False
train_mask = torch.tensor(train_mask)
test_mask = torch.tensor(test_mask)
degree = degree.to(device)
adj = adj.to(device)
data = data.to(device)
features = features.to(device)

In [6]:
class FBLayer(nn.Module):
    def __init__(self, dim_in, dim_out):
      super().__init__()
      self.low = nn.Linear(dim_in, dim_out, bias = False)
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.low.weight, gain)
      self.activation = nn.ReLU()
        
    def forward(self, input, adj, degree):
      lap = degree - adj
      inter = scipy.linalg.fractional_matrix_power(degree.cpu(), (-1/2))
      d_inv = torch.from_numpy(inter).to(device)
      Llp = torch.mm(torch.mm(d_inv, lap), d_inv)
      Hl = torch.mm(Llp, self.activation(self.low(input)))
      return Hl

In [7]:
class FBGCN(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.fb1 = FBLayer(dataset.num_node_features, 16)
      self.fb2 = FBLayer(16 ,dataset.num_classes)
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)
      self.aL_1 = torch.nn.Parameter(torch.rand(1))
      self.aH_1 = torch.nn.Parameter(torch.rand(1))
      self.aL_2 = torch.nn.Parameter(torch.rand(1))
      self.aH_2 = torch.nn.Parameter(torch.rand(1))

    def forward(self, data, adj, degree):
      input, edge_index = data.x, data.edge_index
      x_1 = self.fb1(input, adj, degree) #high pass
      y_1 = self.conv1(input, edge_index) #low pass layer
      z_1 = self.aL_1 * x_1 + self.aH_1 * y_1
      z_1 = F.relu(z_1)
      z_1 = F.dropout(z_1, training=self.training)
      x_2 = self.fb2(z_1, adj, degree)
      y_2 = self.conv2(z_1, edge_index)
      z_2 = self.aL_2 * x_2 + self.aH_2 * y_2

      return F.log_softmax(z_2, dim=1)

In [8]:
model = FBGCN().to(device)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-5)
model.train()
for epoch in range(100):
    print(epoch)
    optimizer.zero_grad()
    torch.autograd.detect_anomaly
    out = model(data, adj, degree)
    loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
    loss.backward()
    optimizer.step()

In [10]:
model.eval()
_, pred = model(data, adj, degree).max(dim=1)
correct = int(pred[test_mask].eq(data.y[test_mask]).sum() .item())
acc = correct / int(test_mask.sum())
print('FBGCN Accuracy: {:.4f}'.format(acc))

FBGCN Accuracy: 0.2725


In [11]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

aL_1 tensor([0.8395], device='cuda:0')
aH_1 tensor([0.3290], device='cuda:0')
aL_2 tensor([1.2091], device='cuda:0')
aH_2 tensor([0.7611], device='cuda:0')
fb1.low.weight tensor([[-4.2313e-02,  1.0933e-02,  1.2751e-02,  ...,  5.7112e-03,
         -2.0937e-01, -7.3691e-02],
        [-1.9001e-01,  1.1278e-01, -2.3552e-02,  ..., -1.8762e-02,
          1.0766e-01, -3.0043e-02],
        [-2.5818e-02, -2.3753e-02,  5.0676e-06,  ...,  6.5787e-04,
          2.6137e-02,  9.2074e-04],
        ...,
        [ 5.5482e-04, -1.8021e-03,  2.8550e-03,  ..., -7.3532e-04,
         -1.0747e-01,  8.5834e-03],
        [ 4.5072e-03,  1.6448e-02, -9.0411e-03,  ...,  2.7899e-04,
         -1.1471e-01,  2.7258e-02],
        [-9.0213e-02, -1.5397e-01,  1.0705e-02,  ..., -2.0560e-03,
         -1.9904e-01,  5.3126e-02]], device='cuda:0')
fb2.low.weight tensor([[ 5.5337e-02, -4.4037e-01, -1.6580e-01, -4.5251e-01,  1.3546e-01,
         -3.1130e-01, -1.0537e-01,  9.0686e-01, -1.8726e-02,  8.1554e-01,
          1.7726e

In [13]:
for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()